In [1]:
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch
import sys
import gym
import numpy as np
import os
import time
import random
from collections import namedtuple
import torch
from tqdm import tqdm,trange
import torch.nn as nn
from torch.optim import Adam
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

### behave Policy

In [14]:
class PolicyNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(PolicyNetwork, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc(x)
        x = self.softmax(x)
        return x

def behavior_policy_gradient(env, behavior_policy, target_policy, num_episodes, max_episode_length, discount_factor, learning_rate):
    episode_rewards = []
    episode_lengths = []

    for _ in range(num_episodes):
        episode_reward = 0
        episode_length = 0
        observation = env.reset()

        for _ in (t:=tqdm(range(max_episode_length))):
            print(observation)
            action = behavior_policy.select_action(observation)
            next_observation, reward, done, _ = env.step(action)
            # 計算行為策略的梯度
            behavior_prob = behavior_policy.get_probability(observation, action)
            target_prob = target_policy.get_probability(observation, action)
            gradient = behavior_policy.get_gradient(observation, action)

            # 更新行為策略的參數
            behavior_policy.update_parameters(gradient, learning_rate)

            episode_reward += reward * importance_sampling_ratio
            episode_length += 1
            
            if done:
                break

            observation = next_observation
            t.set_description(f"Action {action},Importance ratio {importance_sampling_ratio}")
            t.set_postfix(f'Episode reward {episode_reward} reward {reward}')
            

        episode_rewards.append(episode_reward)
        episode_lengths.append(episode_length)

    average_reward = np.mean(episode_rewards)
    average_length = np.mean(episode_lengths)

    return average_reward, average_length

In [11]:
# 定義行為策略和目標策略的類
class BehaviorPolicy:
    def __init__(self, num_actions, num_features):
        self.weights = np.zeros((num_actions, num_features))

    def select_action(self, observation):
        probabilities = self.get_probabilities(observation)
        return np.random.choice(len(probabilities), p=probabilities) ## Select from [2,]

    def get_probabilities(self, observation):
        print(self.weights,observation)
        logits = np.dot(self.weights, observation)
        probabilities = np.exp(logits) / np.sum(np.exp(logits))
        return probabilities

    def get_probability(self, observation, action):
        probabilities = self.get_probabilities(observation)
        return probabilities[action]

    def get_gradient(self, observation, action):
        probabilities = self.get_probabilities(observation)
        gradient = np.zeros_like(self.weights)
        gradient[action] = observation * (1 - probabilities[action])
        return gradient

    def update_parameters(self, gradient, learning_rate):
        self.weights += learning_rate * gradient


class TargetPolicy:
    def __init__(self, num_actions, num_features):
        self.weights = np.zeros((num_actions, num_features))

    def get_probability(self, observation, action):
        logits = np.dot(self.weights, observation)
        probabilities = np.exp(logits) / np.sum(np.exp(logits))
        return probabilities[action]


In [12]:
# 使用範例：
env = gym.make('CartPole-v1')  # 使用CartPole-v1環境
num_episodes = 1000  # 執行1000個回合
max_episode_length = 200  # 每個回合的最大步數
discount_factor = 0.99  # 折扣因子
learning_rate = 0.01  # 學習率

In [27]:
# 執行Data-Efficient Policy Evaluation with Behavior Policy Gradient
num_actions = env.action_space.n
num_features = env.observation_space.shape[0]
print(f'num_actions {num_actions} ,num_features {num_features}')
behavior_policy = BehaviorPolicy(num_actions, num_features)
target_policy = TargetPolicy(num_actions, num_features)

average_reward, average_length = behavior_policy_gradient(env, behavior_policy, target_policy, num_episodes, max_episode_length, discount_factor, learning_rate)

print("Average Reward:", average_reward)
print("Average Episode Length:", average_length)

num_actions 2 ,num_features 4


  0%|          | 0/200 [00:00<?, ?it/s]

(array([ 0.04883818, -0.00864507, -0.02790764, -0.01675023], dtype=float32), {})
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]] (array([ 0.04883818, -0.00864507, -0.02790764, -0.01675023], dtype=float32), {})


ValueError: shapes (2,4) and (2,) not aligned: 4 (dim 1) != 2 (dim 0)